In [2]:
import os
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_experimental.text_splitter import SemanticChunker
from langchain.schema import Document
import gradio as gr
import glob

In [3]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

In [ ]:
file_paths = glob.glob("knowledge_base/**/*.md", recursive=True)

documents = []
for path in file_paths:
    with open(path, "r", encoding="utf-8") as f:
        text = f.read()
        doc_type = os.path.basename(os.path.dirname(path)) 

        documents.append(
            Document(
                page_content=text,
                metadata={
                    "doc_type": doc_type,
                },
            )
        )

In [ ]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small", openai_api_key=api_key)

# text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
text_splitter = SemanticChunker(embeddings)
chunks = text_splitter.split_documents(documents)

print(f"Total number of chunks: {len(chunks)}")

In [ ]:
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory="chroma_db"
)
vectorstore.persist()
print("Chroma vector store built.")

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, openai_api_key=api_key)
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
retriever = vectorstore.as_retriever()
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
)

In [ ]:
query = "Tell me about Langchain."
result = conversation_chain({"question": query})

print("Answer:")
print(result["answer"])

In [ ]:
def rag_chat(query, history):
    response = conversation_chain({"question": query})
    answer = response["answer"]
    return answer

with gr.Blocks(theme=gr.themes.Soft()) as rag_ui:
    gr.Markdown("# RAG Chat Assistant")
    gr.Markdown("Ask questions about your Markdown knowledge base.")
    chat_box = gr.ChatInterface(
        fn=rag_chat,
        title="RAG Knowledge Base Assistant",
        description="Chat with your Markdown-based knowledge base using RAG."
    )

In [ ]:
rag_ui.launch(debug=True, share=True)